In [1]:
import pandas as pd
import dask.dataframe as dd
import praw
from dask.diagnostics import ProgressBar
from tqdm import tqdm
import hashlib
from PIL import Image
import requests
from io import BytesIO
from shared_code.utility.schemas.pyarrow_schema import schema
from shared_code.utility.spark.set_environ import set_azure_env
import os
import torch
from datetime import datetime
import pandas as pd
from pmaw import PushshiftAPI

set_azure_env()

from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter
from shared_code.utility.scripts.blip_caption import BlipCaption

pbar = ProgressBar()
pbar.register()

tqdm.pandas()
tqdm.pandas(desc="global")

from tqdm.dask import TqdmCallback
cb = TqdmCallback(desc="global")
cb.register()

file_system = AzureFileStorageAdapter('data').get_file_storage()

In [2]:
%%time
#"SFWRedheads","sfwpetite","SFWNextDoorGirls",
subs = ["CityPorn","EarthPorn","memes","spaceporn","trippinthroughtime","fatsquirrelhate","oldladiesbakingpies","itookapicture"]

CPU times: total: 0 ns
Wall time: 1 ms


In [3]:
%%time

extant_data = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
display(extant_data)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1000cej,CityPorn,OtterlyFoxy,New York in the fog,cars are parked on the side of the road in the...,7a8d96e378c15c8ab8440ac311f12c11,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,https://i.redd.it/4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,False
1,1000glf,AmIhotAF,toolate_sharkbait,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,e554c1ed7ffa2740436ac082068b2824,/r/AmIhotAF/comments/1000glf/just_looking_for_...,https://i.redd.it/4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,False
2,1000j1n,greentext,trent8051,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,1dec3dabb5e46cde01855d06089c287a,/r/greentext/comments/1000j1n/anon_wants_elon_...,https://i.redd.it/3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,D:\data\images\greentext\3mewbe0wjb9a1.jpg,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19763,ywjp42,itookapicture,leflore4,ITAP - Of coach teaching young men to stand st...,there is a man in a black hat and a baseball c...,f9c5e68a2e6082b0827469253807647e,/r/itookapicture/comments/ywjp42/itap_of_coach...,https://i.redd.it/8e4ku12iq80a1.jpg,f9c5e68a2e6082b0827469253807647e.jpg,D:\data\images\itookapicture\f9c5e68a2e6082b08...,D:\data\images\itookapicture\thumbnail\f9c5e68...,True,False
19764,ywjli7,itookapicture,Naj183,ITAP of a Turkey Vulture spreading its wings,a close up of a large bird with its wings open...,a37c752a3ecdd654cd402280b0a3de7f,/r/itookapicture/comments/ywjli7/itap_of_a_tur...,https://i.redd.it/35zgwn967a0a1.jpg,a37c752a3ecdd654cd402280b0a3de7f.jpg,D:\data\images\itookapicture\a37c752a3ecdd654c...,D:\data\images\itookapicture\thumbnail\a37c752...,True,False
19765,ywjew1,itookapicture,ExaminationHopeful36,ITAP of my cat looking out of the window,a close up of a black cat sitting on a window ...,53f99cb828e71b8a3172fea41d1f2259,/r/itookapicture/comments/ywjew1/itap_of_my_ca...,https://i.redd.it/orn7pcse5a0a1.jpg,53f99cb828e71b8a3172fea41d1f2259.jpg,D:\data\images\itookapicture\53f99cb828e71b8a3...,D:\data\images\itookapicture\thumbnail\53f99cb...,True,False
19766,ywjeai,itookapicture,retrospektor,ITAP of an empty beer bottle,a close up of a candle lit up in a candlelight,9d5acf1a16798238d606fc3646d16f75,/r/itookapicture/comments/ywjeai/itap_of_an_em...,https://i.redd.it/oyuscvpg5a0a1.jpg,9d5acf1a16798238d606fc3646d16f75.jpg,D:\data\images\itookapicture\9d5acf1a16798238d...,D:\data\images\itookapicture\thumbnail\9d5acf1...,True,False


CPU times: total: 766 ms
Wall time: 3.33 s


In [4]:
reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA',
								  user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

api: PushshiftAPI = PushshiftAPI(praw=reddit, num_workers=12)
start_date: int = int(datetime.strptime('2022-01-01', '%Y-%m-%d').timestamp())

Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


In [5]:
def filter_by_image_post(item) -> bool:
	return not (item['selftext'].__contains__('[removed]') or item['selftext'].__contains__('[deleted]') and not item[
		'url'].endswith('.jpg'))

In [6]:
def fetch_image(x: object) -> object:
	try:
		url = x['original_url']
		subreddit = x['subreddit']
		response = requests.get(url)
		md5 = hashlib.md5(response.content).hexdigest()
		os.makedirs(f"D:\\data\\images\\{subreddit}", exist_ok=True)
		out_path = f"D:\\data\\images\\{subreddit}\\{md5}.jpg"

		if os.path.exists(out_path):
			return out_path

		if md5 in list(extant_data['hash']):
			return out_path

		if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
			raw_image = Image.open(BytesIO(response.content))
			raw_image.save(out_path)
			raw_image.close()
			return out_path
		else:
			return ""
	except Exception as e:
		# print(e)
		return ""

In [7]:
def make_thumbnail(x: object) -> str:
	image_path = x['path']
	md5 = x['hash']
	subreddit = x['subreddit']
	os.makedirs(f"D:\\data\\images\\{subreddit}\\thumbnail", exist_ok=True)
	out_path = f"D:\\data\\images\\{subreddit}\\thumbnail\\{md5}.jpg"
	if not os.path.exists(image_path):
		return ""
	if os.path.exists(out_path):
		return out_path
	img = Image.open(image_path)
	try:
		copied_image = img.copy()
		result_copy = copied_image.resize((512, 512))
		result_copy.save(out_path)
		result_copy.close()
		img.close()
		return out_path
	except Exception as e:
		# print(e)
		return ""
	finally:
		img.close()

In [8]:
def exists(x: object) -> bool:
	try:
		image_path = os.path.exists(x['path'])
		thumbnail_path = os.path.exists(x['thumbnail_path'])
		return image_path and thumbnail_path
	except Exception as e:
		# print(e)
		return False

In [9]:
def split_hash(x: object) -> str:
	try:
		name = x['image_name']
		return name.split('.')[0]
	except Exception as e:
		# print(e)
		return ""

In [10]:
def caption_image(x: object, blip_captioning: BlipCaption) -> str:
	try:
		path = x['path']
		exists_image = bool(x['exists'])
		resulting_caption = x['caption']

		if not os.path.exists(path):
			return ""

		if not exists_image:
			return ""

		if len(resulting_caption) > 5:
			return resulting_caption

		resulting_caption = blip_captioning.caption_image(path)
		return resulting_caption

	except Exception as e:
		return ""
	finally:
		pass

In [11]:
def get_image_name(x: object) -> str:
	try:
		path = x['path']
		if path == "":
			return ""
		return os.path.basename(path)
	except Exception as e:
		# print(e)
		return ""

In [12]:
%%time

all_posts = []

for item in subs:
	posts = api.search_submissions(search_window=365, subreddit=item, limit=1000, filter_fn=filter_by_image_post, mem_safe=True, cache_dir='D:\\cache\\')
	[ all_posts.append(post) for post in posts ]

CPU times: total: 30 s
Wall time: 7min 47s


In [13]:
%%time

df = pd.DataFrame([item for item in all_posts])
display(df)

,comment_limit,comment_sort,_reddit,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,...,_fetched,_comments_by_id,link_flair_template_id,crosspost_parent_list,crosspost_parent,author_cakeday,is_gallery,media_metadata,gallery_data,poll_data
0,2048,confidence,<praw.reddit.Reddit object at 0x00000242616E10A0>,None,CityPorn,,t2_xy5rp,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2048,confidence,<praw.reddit.Reddit object at 0x00000242616E10A0>,None,CityPorn,,t2_97hh49l1,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2048,confidence,<praw.reddit.Reddit object at 0x00000242616E10A0>,None,CityPorn,,t2_t2wcr4zn,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2048,confidence,<praw.reddit.Reddit object at 0x00000242616E10A0>,None,CityPorn,,t2_mvttb7jg,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2048,confidence,<praw.reddit.Reddit object at 0x00000242616E10A0>,None,CityPorn,,t2_74uby,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5318,2048,confidence,<praw.reddit.Reddit object at 0x00000242674D4940>,None,itookapicture,,t2_5zpnd38m,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5319,2048,confidence,<praw.reddit.Reddit object at 0x00000242674D4940>,None,itookapicture,,t2_3l98q7vy,False,None,0,...,False,{},f16cd2a8-621d-11e6-a75b-0eaaaf3b3d9d,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5320,2048,confidence,<praw.reddit.Reddit object at 0x00000242674D4940>,None,itookapicture,,t2_1rjdre0w,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5321,2048,confidence,<praw.reddit.Reddit object at 0x00000242674D4940>,None,itookapicture,,t2_6e45t8f6,False,None,0,...,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


CPU times: total: 594 ms
Wall time: 789 ms


In [14]:
%%time

initial = pd.DataFrame({}, columns=schema.names)
initial['id'] = df['id']
initial['subreddit'] = df['subreddit']
initial['author'] = df['author']
initial['title'] = df['title']
initial['caption'] = ""
initial['hash'] = ""
initial['permalink'] = df['permalink']
initial['original_url'] = df['url']
initial['image_name'] = ""
initial['path'] = ""
initial['thumbnail_path'] = ""
initial['exists'] = False
initial['curated'] = False

filtered_initial = initial.where(initial['original_url'].str.endswith('.jpg')).dropna(how='all').reset_index().drop('index', axis=1)

display(filtered_initial)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,114fd0m,CityPorn,Saltedline,"Yashio Residential Complex, Shinagawa, Tokyo",,,/r/CityPorn/comments/114fd0m/yashio_residentia...,https://i.imgur.com/QXJ78B6.jpg,,,,False,False
1,114dsn9,CityPorn,SumfinityPhoto,Mädlerpassage in Leipzig - Home to Auerbachs K...,,,/r/CityPorn/comments/114dsn9/mädlerpassage_in_...,https://i.redd.it/yjb5k2bpdpia1.jpg,,,,False,False
2,1149ggo,CityPorn,LbamaGaming,"Seattle, Washington, USA",,,/r/CityPorn/comments/1149ggo/seattle_washingto...,https://i.redd.it/diojclrtlpia1.jpg,,,,False,False
3,11494fv,CityPorn,Pittsburgh_Photos,"This use to be a parking lot. Schenley Plaza, ...",,,/r/CityPorn/comments/11494fv/this_use_to_be_a_...,https://i.redd.it/gdpd814ripia1.jpg,,,,False,False
4,11493wd,CityPorn,Surrealplaces,"Calgary, Canada",,,/r/CityPorn/comments/11493wd/calgary_canada/,https://i.redd.it/rxt7w6f31oia1.jpg,,,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4040,10r04ji,itookapicture,RalphieLils28,ITAP of a swan,,,/r/itookapicture/comments/10r04ji/itap_of_a_swan/,https://i.redd.it/dbsr1mczinfa1.jpg,,,,False,False
4041,10r02f5,itookapicture,DigitalDavid94,ITAP of a witch,,,/r/itookapicture/comments/10r02f5/itap_of_a_wi...,https://i.redd.it/myn7gm5linfa1.jpg,,,,False,False
4042,10qzp5z,itookapicture,giveneric,ITAP of a Bald Eagle,,,/r/itookapicture/comments/10qzp5z/itap_of_a_ba...,https://i.redd.it/m62h1tq5gnfa1.jpg,,,,False,False
4043,10qzmcp,itookapicture,ankitdb11,ITAP of flamingos.,,,/r/itookapicture/comments/10qzmcp/itap_of_flam...,https://i.redd.it/xxgisiclfnfa1.jpg,,,,False,False


CPU times: total: 188 ms
Wall time: 315 ms


In [15]:
%%time

new_entries = filtered_initial.where(~filtered_initial['id'].isin(extant_data['id'])).dropna(how='all').reset_index().drop('index', axis=1)

display(new_entries)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1144tsr,CityPorn,egnogra,Neo Goldlen Gai 新宿ゴールデン街,,,/r/CityPorn/comments/1144tsr/neo_goldlen_gai_新...,https://i.redd.it/56m7bhauhoia1.jpg,,,,False,False
1,11v1wnw,CityPorn,HamSolo14,"São Paulo Historical Center, Brasil",,,/r/CityPorn/comments/11v1wnw/são_paulo_histori...,https://i.redd.it/j1yn5isj0moa1.jpg,,,,False,False
2,11v17te,CityPorn,gabigolterasga,"Santiago, Chile. The city with the Highest HDI...",,,/r/CityPorn/comments/11v17te/santiago_chile_th...,https://i.redd.it/08jhmp5zdkoa1.jpg,,,,False,False
3,11uyetd,CityPorn,atzucach,Barcelona,,,/r/CityPorn/comments/11uyetd/barcelona/,https://i.redd.it/rzrdrgpjfloa1.jpg,,,,False,False
4,11uww70,CityPorn,Beatnik64,"San Francisco, California",,,/r/CityPorn/comments/11uww70/san_francisco_cal...,https://i.redd.it/16vgdp5qpjoa1.jpg,,,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1688,10r04ji,itookapicture,RalphieLils28,ITAP of a swan,,,/r/itookapicture/comments/10r04ji/itap_of_a_swan/,https://i.redd.it/dbsr1mczinfa1.jpg,,,,False,False
1689,10r02f5,itookapicture,DigitalDavid94,ITAP of a witch,,,/r/itookapicture/comments/10r02f5/itap_of_a_wi...,https://i.redd.it/myn7gm5linfa1.jpg,,,,False,False
1690,10qzp5z,itookapicture,giveneric,ITAP of a Bald Eagle,,,/r/itookapicture/comments/10qzp5z/itap_of_a_ba...,https://i.redd.it/m62h1tq5gnfa1.jpg,,,,False,False
1691,10qzmcp,itookapicture,ankitdb11,ITAP of flamingos.,,,/r/itookapicture/comments/10qzmcp/itap_of_flam...,https://i.redd.it/xxgisiclfnfa1.jpg,,,,False,False


CPU times: total: 109 ms
Wall time: 163 ms


In [16]:
%%time

new_entries_1 = new_entries.copy()

tqdm.pandas(desc="Fetch-Image")
with ProgressBar():
	new_entries_1['path'] = new_entries_1.progress_apply(lambda x: fetch_image(x), axis=1)

tqdm.pandas(desc="Get-Image-Name")
with ProgressBar():
	new_entries_1['image_name'] = new_entries_1.progress_apply(lambda x: get_image_name(x), axis=1)

tqdm.pandas(desc="Split-Hash")
with ProgressBar():
	new_entries_1['hash'] = new_entries_1.progress_apply(lambda x: split_hash(x), axis=1)

display(new_entries_1)

Split-Hash: 100%|██████████| 1693/1693 [00:00<00:00, 17453.37it/s]


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1144tsr,CityPorn,egnogra,Neo Goldlen Gai 新宿ゴールデン街,,9c726e3ad6114dd06a25ae0e9d3dd9ba,/r/CityPorn/comments/1144tsr/neo_goldlen_gai_新...,https://i.redd.it/56m7bhauhoia1.jpg,9c726e3ad6114dd06a25ae0e9d3dd9ba.jpg,D:\data\images\CityPorn\9c726e3ad6114dd06a25ae...,,False,False
1,11v1wnw,CityPorn,HamSolo14,"São Paulo Historical Center, Brasil",,57cfe2d8e6229af18cca2fc497ba9a1b,/r/CityPorn/comments/11v1wnw/são_paulo_histori...,https://i.redd.it/j1yn5isj0moa1.jpg,57cfe2d8e6229af18cca2fc497ba9a1b.jpg,D:\data\images\CityPorn\57cfe2d8e6229af18cca2f...,,False,False
2,11v17te,CityPorn,gabigolterasga,"Santiago, Chile. The city with the Highest HDI...",,c02b0059628ef836c1c5f883fbc0fd66,/r/CityPorn/comments/11v17te/santiago_chile_th...,https://i.redd.it/08jhmp5zdkoa1.jpg,c02b0059628ef836c1c5f883fbc0fd66.jpg,D:\data\images\CityPorn\c02b0059628ef836c1c5f8...,,False,False
3,11uyetd,CityPorn,atzucach,Barcelona,,54936a6255c7c3df757ecef506f786d4,/r/CityPorn/comments/11uyetd/barcelona/,https://i.redd.it/rzrdrgpjfloa1.jpg,54936a6255c7c3df757ecef506f786d4.jpg,D:\data\images\CityPorn\54936a6255c7c3df757ece...,,False,False
4,11uww70,CityPorn,Beatnik64,"San Francisco, California",,43d548a485831050458f3c71dc6109d1,/r/CityPorn/comments/11uww70/san_francisco_cal...,https://i.redd.it/16vgdp5qpjoa1.jpg,43d548a485831050458f3c71dc6109d1.jpg,D:\data\images\CityPorn\43d548a485831050458f3c...,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1688,10r04ji,itookapicture,RalphieLils28,ITAP of a swan,,b9d9119d89c2473df5828a15db9f5264,/r/itookapicture/comments/10r04ji/itap_of_a_swan/,https://i.redd.it/dbsr1mczinfa1.jpg,b9d9119d89c2473df5828a15db9f5264.jpg,D:\data\images\itookapicture\b9d9119d89c2473df...,,False,False
1689,10r02f5,itookapicture,DigitalDavid94,ITAP of a witch,,17fc7087b047d9e5b5da9ce299077706,/r/itookapicture/comments/10r02f5/itap_of_a_wi...,https://i.redd.it/myn7gm5linfa1.jpg,17fc7087b047d9e5b5da9ce299077706.jpg,D:\data\images\itookapicture\17fc7087b047d9e5b...,,False,False
1690,10qzp5z,itookapicture,giveneric,ITAP of a Bald Eagle,,a7205f636ae0be5b3c6dd1f5e73ab7ee,/r/itookapicture/comments/10qzp5z/itap_of_a_ba...,https://i.redd.it/m62h1tq5gnfa1.jpg,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,D:\data\images\itookapicture\a7205f636ae0be5b3...,,False,False
1691,10qzmcp,itookapicture,ankitdb11,ITAP of flamingos.,,827260f27293bf219f260ed275534368,/r/itookapicture/comments/10qzmcp/itap_of_flam...,https://i.redd.it/xxgisiclfnfa1.jpg,827260f27293bf219f260ed275534368.jpg,D:\data\images\itookapicture\827260f27293bf219...,,False,False


CPU times: total: 8min 20s
Wall time: 16min 8s


In [17]:
%%time

new_entries_2 = new_entries_1.copy()

tqdm.pandas(desc="Create-Thumbnail")
with ProgressBar():
	new_entries_2['thumbnail_path'] = new_entries_2.progress_apply(lambda x: make_thumbnail(x), axis=1)

tqdm.pandas(desc="Checking-Exists")
with ProgressBar():
	new_entries_2['exists'] = new_entries_2.progress_apply(lambda x: exists(x), axis=1)

display(new_entries_2)

Checking-Exists: 100%|██████████| 1693/1693 [00:01<00:00, 1674.75it/s]


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1144tsr,CityPorn,egnogra,Neo Goldlen Gai 新宿ゴールデン街,,9c726e3ad6114dd06a25ae0e9d3dd9ba,/r/CityPorn/comments/1144tsr/neo_goldlen_gai_新...,https://i.redd.it/56m7bhauhoia1.jpg,9c726e3ad6114dd06a25ae0e9d3dd9ba.jpg,D:\data\images\CityPorn\9c726e3ad6114dd06a25ae...,D:\data\images\CityPorn\thumbnail\9c726e3ad611...,True,False
1,11v1wnw,CityPorn,HamSolo14,"São Paulo Historical Center, Brasil",,57cfe2d8e6229af18cca2fc497ba9a1b,/r/CityPorn/comments/11v1wnw/são_paulo_histori...,https://i.redd.it/j1yn5isj0moa1.jpg,57cfe2d8e6229af18cca2fc497ba9a1b.jpg,D:\data\images\CityPorn\57cfe2d8e6229af18cca2f...,D:\data\images\CityPorn\thumbnail\57cfe2d8e622...,True,False
2,11v17te,CityPorn,gabigolterasga,"Santiago, Chile. The city with the Highest HDI...",,c02b0059628ef836c1c5f883fbc0fd66,/r/CityPorn/comments/11v17te/santiago_chile_th...,https://i.redd.it/08jhmp5zdkoa1.jpg,c02b0059628ef836c1c5f883fbc0fd66.jpg,D:\data\images\CityPorn\c02b0059628ef836c1c5f8...,D:\data\images\CityPorn\thumbnail\c02b0059628e...,True,False
3,11uyetd,CityPorn,atzucach,Barcelona,,54936a6255c7c3df757ecef506f786d4,/r/CityPorn/comments/11uyetd/barcelona/,https://i.redd.it/rzrdrgpjfloa1.jpg,54936a6255c7c3df757ecef506f786d4.jpg,D:\data\images\CityPorn\54936a6255c7c3df757ece...,D:\data\images\CityPorn\thumbnail\54936a6255c7...,True,False
4,11uww70,CityPorn,Beatnik64,"San Francisco, California",,43d548a485831050458f3c71dc6109d1,/r/CityPorn/comments/11uww70/san_francisco_cal...,https://i.redd.it/16vgdp5qpjoa1.jpg,43d548a485831050458f3c71dc6109d1.jpg,D:\data\images\CityPorn\43d548a485831050458f3c...,D:\data\images\CityPorn\thumbnail\43d548a48583...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1688,10r04ji,itookapicture,RalphieLils28,ITAP of a swan,,b9d9119d89c2473df5828a15db9f5264,/r/itookapicture/comments/10r04ji/itap_of_a_swan/,https://i.redd.it/dbsr1mczinfa1.jpg,b9d9119d89c2473df5828a15db9f5264.jpg,D:\data\images\itookapicture\b9d9119d89c2473df...,D:\data\images\itookapicture\thumbnail\b9d9119...,True,False
1689,10r02f5,itookapicture,DigitalDavid94,ITAP of a witch,,17fc7087b047d9e5b5da9ce299077706,/r/itookapicture/comments/10r02f5/itap_of_a_wi...,https://i.redd.it/myn7gm5linfa1.jpg,17fc7087b047d9e5b5da9ce299077706.jpg,D:\data\images\itookapicture\17fc7087b047d9e5b...,D:\data\images\itookapicture\thumbnail\17fc708...,True,False
1690,10qzp5z,itookapicture,giveneric,ITAP of a Bald Eagle,,a7205f636ae0be5b3c6dd1f5e73ab7ee,/r/itookapicture/comments/10qzp5z/itap_of_a_ba...,https://i.redd.it/m62h1tq5gnfa1.jpg,a7205f636ae0be5b3c6dd1f5e73ab7ee.jpg,D:\data\images\itookapicture\a7205f636ae0be5b3...,D:\data\images\itookapicture\thumbnail\a7205f6...,True,False
1691,10qzmcp,itookapicture,ankitdb11,ITAP of flamingos.,,827260f27293bf219f260ed275534368,/r/itookapicture/comments/10qzmcp/itap_of_flam...,https://i.redd.it/xxgisiclfnfa1.jpg,827260f27293bf219f260ed275534368.jpg,D:\data\images\itookapicture\827260f27293bf219...,D:\data\images\itookapicture\thumbnail\827260f...,True,False


CPU times: total: 8min 35s
Wall time: 11min


In [18]:
%%time

blip = BlipCaption(1)

CPU times: total: 1min 7s
Wall time: 1min 17s


In [ ]:
%%time

new_entries_final = new_entries_2.copy()
torch.cuda.empty_cache()

with pbar:
	ddf = dd.from_pandas(new_entries_final, npartitions=6)
	new_entries_final['caption'] = ddf.apply(lambda x: caption_image(x, blip), meta=('str', object), axis=1).compute()

torch.cuda.empty_cache()

display(new_entries_final)

[                                        ] | 0% Completed | 790.10 us

global:   0%|          | 0/6 [00:00<?, ?it/s]

[                                        ] | 0% Completed | 14.97 sms

D:\workspaces\General\venv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[                                        ] | 0% Completed | 27m 8sss

In [ ]:
%%time

tqdm.pandas(desc="Filtering")
writeable_entries = new_entries_final\
	.where(~new_entries_final['id'].isin(extant_data['id']))\
	.dropna(how='all')\
	.reset_index()\
	.drop('index', axis=1)

display(writeable_entries)

In [ ]:
%%time

combined_result = pd.concat([extant_data, writeable_entries])

display(combined_result)

In [ ]:
combined_result.reindex()
combined_result.to_csv('processed_raw_data.csv', index=False)
f = AzureFileStorageAdapter('data').get_file_storage()
f.put('processed_raw_data.csv','data/processed_raw_data.csv', overwrite=True)

In [ ]:
final = pd.read_csv(f.open('data/processed_raw_data.csv'))
final.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

In [ ]:
final = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system)
display(final)